In [7]:
import numpy as np


class AckleyTF:
    def __init__(
        self,
        constrain=False,
    ):
        self.bounds = [-32, 32]
        self.n_dim = 2
        self.min = [0, 0]
        self.fmin = 0
        self.max = [32, 32]
        self.fmax = 22.3497
        
        self.grid_size = self.bounds[1] - self.bounds[0] + 1
        self._tensor_constraint = None
        if constrain:
            self._build_constraint()

    def _build_constraint(self) -> None:
        X, Y = np.meshgrid(
            np.arange(self.bounds[0], self.bounds[1] + 1),
            np.arange(self.bounds[0], self.bounds[1] + 1)
        )
        R_squared = X**2 + Y**2
        self._tensor_constraint = (R_squared < 10**2).astype(int)

    def _coord_to_index(self, x):
        return [int(xi - self.bounds[0]) for xi in x]

    def _index_to_coord(self, idx):
        return [int(i + self.bounds[0]) for i in idx]

    def function(self, x, y):
        a, b, c = 20, 0.2, 2*np.pi
        d = 2
        xy = np.array([x, y])
        sum1 = -a * np.exp(-b * np.sqrt(np.sum(xy ** 2) / d))
        sum2 = -np.exp(np.sum(np.cos(c * xy)) / d)
        return sum1 + sum2 + a + np.exp(1)

    def evaluate(self, x):
        if not self.is_dimensionality_valid(x) or not self.is_in_bounds(x):
            return self.fmax
        
        return self.function(x[0], x[1])

    def sample_violation_indices(self, num_samples: int) -> np.ndarray:
        if self._tensor_constraint is None:
            raise ValueError("Constraint not initialized")
        
        indices = np.array(np.where(self._tensor_constraint == 0)).T
        if num_samples > len(indices):
            raise ValueError("num_samples is too large")
            
        return indices[np.random.choice(len(indices), size=num_samples, replace=False)]

    def sample_violation_path(self, num_samples: int = 200) -> list[tuple[int, int]]:
        random_indices = self.sample_violation_indices(num_samples)
        return [tuple(self._index_to_coord(idx)) for idx in random_indices]

    def is_dimensionality_valid(self, x):
        return len(x) == 2

    def is_in_bounds(self, x):
        return all(self.bounds[0] <= xi <= self.bounds[1] for xi in x)

In [8]:
obj = AckleyTF(constrain=True)

In [9]:
obj._tensor_constraint

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [6]:
obj._tensor_constraint.sum()

np.int64(69)

In [5]:
categories = list(range(-32, 33))
print(categories)

[-32, -31, -30, -29, -28, -27, -26, -25, -24, -23, -22, -21, -20, -19, -18, -17, -16, -15, -14, -13, -12, -11, -10, -9, -8, -7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]


In [10]:
const = obj._tensor_constraint

In [11]:
const[20, -25]

np.int64(0)

In [12]:
X, Y = np.meshgrid(
    np.arange(-2, 3),
    np.arange(-2, 3)
)

X, Y

(array([[-2, -1,  0,  1,  2],
        [-2, -1,  0,  1,  2],
        [-2, -1,  0,  1,  2],
        [-2, -1,  0,  1,  2],
        [-2, -1,  0,  1,  2]]),
 array([[-2, -2, -2, -2, -2],
        [-1, -1, -1, -1, -1],
        [ 0,  0,  0,  0,  0],
        [ 1,  1,  1,  1,  1],
        [ 2,  2,  2,  2,  2]]))

In [13]:
X ** 2, Y ** 2

(array([[4, 1, 0, 1, 4],
        [4, 1, 0, 1, 4],
        [4, 1, 0, 1, 4],
        [4, 1, 0, 1, 4],
        [4, 1, 0, 1, 4]]),
 array([[4, 4, 4, 4, 4],
        [1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [4, 4, 4, 4, 4]]))

In [14]:
X ** 2 + Y ** 2

array([[8, 5, 4, 5, 8],
       [5, 2, 1, 2, 5],
       [4, 1, 0, 1, 4],
       [5, 2, 1, 2, 5],
       [8, 5, 4, 5, 8]])

In [15]:
X ** 2 + Y ** 2 < 2

array([[False, False, False, False, False],
       [False, False,  True, False, False],
       [False,  True,  True,  True, False],
       [False, False,  True, False, False],
       [False, False, False, False, False]])

In [16]:
obj.evaluate([-1, 1])

np.float64(3.6253849384403627)